## two minor model writing/reading bugs in docplex 

Two bugs, both minor but worth reporting.
 1. If I build a model with the RHS having `float("inf")` the .lp that is exported is unreadable. 
 1. When I export a model as a .sav file, it doesn't export into the current directory but uses some funny path instead.

In [1]:
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader

In [2]:
m = Model("has an infinity on rhs")
v1 = m.continuous_var(ub = 10, name = "v1")
v2 = m.continuous_var(ub = 10, name = "v2")
m.add_constraint(3*v1 + 2*v2 <= float("inf"))
m.maximize(5*v1 + 4*v2)
print m.solve()

solution for: has an infinity on rhs
objective: 90
v1=10.000
v2=10.000



In [3]:
file1 = m.export_as_lp("has_infiniy.lp")
file2 = m.export_as_sav("has_infinity.sav")

**Bug 1** - the `export_as_sav` doesn't use the current working directory.

In [4]:
import os
os.path.exists(os.path.join(os.getcwd(), "has_infiniy.lp"))

True

In [5]:
os.path.exists(os.path.join(os.getcwd(), "has_infinity.sav"))

False

In [6]:
file2

'/var/folders/k_/2mp70w997hj84hxfxkwtd_jm0000gn/T/has_infinity.sav'

** Bug 2 ** It can't read the file that was written. The 'inf' is tripping it up.

In [7]:
mr =ModelReader()
mr.read_model(file1)

*CPLEX error CPLEX Error  1615: Line 8: Expected number, found 'i'.
 reading file has_infiniy.lp - exiting


The save format does work here.

In [8]:
m2 = mr.read_model(file2)

In [9]:
m2.get_constraint_by_index(0)

docplex.mp.linear.LinearConstraint[_c1](3v1+2v2,LE,100000000000000000000)

Just to make this totally clear, if I remove the infinity, then the .lp file writing/reading works.

In [10]:
m = Model("no infinity")
v1 = m.continuous_var(ub = 10, name = "v1")
v2 = m.continuous_var(ub = 10, name = "v2")
m.add_constraint(3*v1 + 2*v2 <= 10000)
m.maximize(5*v1 + 4*v2)
print m.solve()

solution for: no infinity
objective: 90
v1=10.000
v2=10.000



In [11]:
file1 = m.export_as_lp("no_infiniy.lp")

In [12]:
m2 = mr.read_model(file1)